In [1]:
import pandas as pd
from bs4 import BeautifulSoup
key ='Your Key'
token = "Seller's token"
devid = 'Your DevId'
certid = 'Your CertId'
from ebaysdk.finding import Connection as finding
from ebaysdk.trading import Connection as trading


### Get ItemID for products in ebay store

In [222]:
# Define API 
api = trading(appid= key, devid=devid, certid=certid,  token=token, config_file=None)
# Get number of pages to load
response = api.execute('GetMyeBaySelling', {'ActiveList':{'Sort':'TimeLeft', 'PaginationResult': True}})
pages = int(response.dict()['ActiveList']['PaginationResult']['TotalNumberOfPages']) + 1
print "Total pages to load:", pages

# Get ProductID for items in ebay store
productids = []
for page in range(1,pages):
    response = api.execute('GetMyeBaySelling', {'ActiveList':{'Sort':'TimeLeft', 'Pagination':{'PageNumber': str(page)}}, 'DetailLevel': 'ReturnAll'})
    ebayoutput = response.dict()['ActiveList']['ItemArray']['Item']

    for product in ebayoutput:
        productids.append(product['ItemID'])

# Check total number of items in store
print "Total items for sale:", len(set(productids))

Total pages to load: 10
Total items for sale: 1669


### Get MPN and UPC for products in ebay store

In [5]:
# Define API 
api = trading(appid= key, devid=devid, certid=certid,  token=token, config_file=None)

# Get MPN and UPC numbers for each item in ebay store
mpn = []
upc = []
for productid in productids:
    response = api.execute('GetItem', {'ItemID': productid, 'DetailLevel': 'ReturnAll'})
    
    # get upc number
    upc_number = response.dict()['Item']['ProductListingDetails']['UPC']
    upc.append(upc_number)
    
    # get mpn number from Item Description (not working for all sellers)
    page = response.dict()['Item']['Description']
    soup = BeautifulSoup(page, "lxml")
    try:
        brand = soup.find_all('li')[0].text.split(':')[1].replace(u'\xa0', ' ')[1:]
    except IndexError:
        brand = soup.find_all('b')[0].text.replace(u'\xa0', ' ')[1:]
        
    try:
        style = soup.find_all('li')[1].text.split(':')[1].replace(u'\xa0', ' ')
    except IndexError:
        style = soup.find_all('b')[1].text.replace(u'\xa0', ' ')
    
    try: 
        color = soup.find_all('li')[2].text.split(':')[1].replace(u'\xa0', ' ')
    except IndexError:
        color = soup.find_all('b')[2].text.replace(u'\xa0', ' ')
    
    product_mpn = brand + style + color
    product_mpn= str(product_mpn.replace(u'\xa0', ' ').replace(u'\xdc', ' ')\
                     .replace('&', '').replace('  ', ' ').replace('Dior Dior', 'Dior')\
                     .replace('   ', ' ').replace('-', ' ').split(',')[0])
    print product_mpn

    mpn.append(product_mpn)


Costa Del Mar Anaa ANA 105 OCP 580P
Christian Dior CD 3252 2X5
Christian Dior Lady 2R GRVK2
Gucci GG0105S 006
Alexander McQueen AMQ 4147/N/S 2IK CC
PRADA SPR 30R UBT 4O2
Alexander McQueen AMQ 4212/S 284 HD
Gucci GG0002S 002
DITA Cooper DRX 2075 B AMB GLD
Miu Miu VMU51N 7S0 1O1
Tiffany Co. TF2134 B 8134
Gucci GG 3789 LWF
MCM 607S 634
DITA Statesman Two DRX 2051 E T TRT GUN
Gucci GG0018O 007
DITA Eclipse 22021 A BLK
Gucci GG 3802/S NIEED
Burberry B 2144 3424
Burberry B 2223 3013
Gucci GG0007O 004
Gucci GG0097S 001
Gucci GG0013O 003
Gucci GG0097S 002
Burberry B 2232 3606
Gucci GG0013O 004
Burberry B 2226 3605
Alexander McQueen AMQ 4212/S 807 DX
Gucci GG0025O 001
Gucci GG0053/S 006
Burberry B 2231 3608
Gucci GG0061S 004
Costa Del Mar CC 21 OGP 580P
Gucci GG0092S 003
Gucci GG0012O 002
Burberry B 2180 3507
Gucci GG0104O 003
Gucci GG0103O 003
Gucci GG 3586/S X1ABN
Gucci GG0093O 002
Gucci GG0018OA 003
Gucci GG0012OA 001
Gucci GG 0042OA 002
Oliver Peoples Rowan BK/SYC
Oliver Peoples OV1107T 512

Saint Laurent Classic 11 008
Oakley OX5100 WINGFOLD 0454
Saint Laurent Classic 11 RAINBOW 001
Cazal  644 65
Saint Laurent Classic 11 RAINBOW 002
Persol 3013 V 95
Miu Miu SMU 54S ZVN AD6
Oakley OX3187 Conductor 0.5 0251
Cazal  663/3 11
Jimmy Choo INES/S EQEHD
Gucci GG0006O 007
Oakley OX8097 Steel Line S 0352
Oakley OX5120 Lizard 2 0354
Jimmy Choo ALLY/S MY2IC
Oakley OX8047 Milestone 2.0 0152
Gucci GG0031O 001
Ray Ban RB 4224 650/6Q
Ray Ban RB 3524 029/71
Ray Ban RB 3025 J M 171 Camouflage
Tiffany Co. TF2117 B 8001
BOSS Hugo Boss 0666/S TU4HD
Christian Dior Reflected TUZKU
Jimmy Choo PATTY/F/S 19THD
TOM FORD TF 5356 048
Ray Ban RB 2140 1187
PRADA VPS 57E TIG 1O1
BOSS Hugo Boss 0440/S QZT72
Ray Ban RB 4223 V 2077
Ray Ban RB 5184 5516
BOSS Hugo Boss 0735/S K8FHD
Oakley OX8037 Crosslink 0352
Ray Ban RB 4228 6185/88
BOSS Hugo Boss 0617/S MJQHA
Ray Ban RB 6317 2834
Oakley OX8041 Crosslink 0156
TOM FORD TF 5356 001
BOSS Hugo Boss 0321/S YCHVW
TOM FORD TF 5347 089
Christian Dior CD 3273 F 3GV
C

Gucci GG 3651/S ANWYR
Alain Mikli A01309 1001
Gucci GG 3804/S CSA9O
Gucci GG 3644/S DWJHA
Balenciaga BA 5064 50B
Tom Ford ANNA 02 TF575 01B
Maui Jim Freight Trains MJ326 02 Black
Gucci GG0013O 002
Alain Mikli AL1408 0101 1520
Alain Mikli A01309 0101
Ic! Berlin 115 Cunostr Black
Ic! Berlin 115 Cunostr Black
Alain Mikli AL 1070 0002
Carrera 92/S NCWHA
Alain Mikli A01308 0101 4320
Tom Ford Sean TF 536 28C
Tom Ford Adrenne TF517 52G
Tom Ford Sean TF 536 28Z
Ic! Berlin Bernardino Gunmetal
Tom Ford Rachel TF 533 01B
Gucci GG 3162/S D28BN
Tom Ford Dashel TF508 08Z
Gucci GG 3817/S RKQDX
Tom Ford Dashel TF508 28Z
Tag Heuer TH 8103 007
Gucci GG 3827/S KCLHD
Tom Ford Adrenne TF517 56Z
Christian Dior Origins1 06J0Z
Marc Jacobs Marc 168/S RHLIR
Fendi FF 0190/S DDBO7
Gucci GG 3717 INL
Gucci GG 3791/S OHNJD
Gucci GG 4266/S J5GJD
Marc Jacobs Marc 165/S LHFIC
Gucci GG 4253/S KJ14X
Marc Jacobs Marc 162/S 807 9P
Gucci GG 3671/S 6UBEU
Christian Dior Chicago2STR SUTHA
Christian Dior Split 1 010KU
Tom Ford 

Gucci GG0117S 003
Gucci GG 3573 X4L
Roberto Cavalli  Cerreto 1042 28C
Gucci GG0118S 002
Roberto Cavalli Sargas 959 092
TOM FORD TF 5411 052
Gucci GG 3789 LWD
Roberto Cavalli Turais 993S D 01B
Roberto Cavalli Tania 979S 01B
TOM FORD TF 5412 001
Persol 3072 S 95/31
TOM FORD TF 5353 001
TOM FORD TF 5365 005
Gucci GG0117S 002
Montblanc MB 406S 05N
Tiffany Co. TF1127 6124
TOM FORD TF 5401 001
TOM FORD TF 5412 052
Gucci GG0117S 001
TOM FORD TF 5282 005
Caviar M6855 LIMITED EDITION C55
TOM FORD TF 5392 020
TOM FORD TF 5401 052
TOM FORD TF 5378 001
Gucci GG0049S 003
Caviar M6854 C55
Caviar M6854 LIMITED EDITION C21
Tiffany Co. TF3053 6108/6V
Caviar M6855 LIMITED EDITION C21
TOM FORD TF 5460 001
Tom Ford Elena TF437 05R
Tiffany Co. TF2149 8134
Marc Jacobs Marc 9/S 25K FU
Ray Ban RB7038 5456
Gucci GG 1091/S 1VDIH
Tom Ford Cliff TF 450 28P
Caviar M 4870 C 24
Tom Ford April TF 393 28P
Bvlgari 8130 H B 501/8G
Caviar M 6173 C 24
Caviar M 3011 C 24
PRADA VPR 14S NAI 1O1
Caviar M 2345 C 24
PRADA SPR 5

### Check Results for MPN and UPC

In [6]:
print "Total MPN's:", len(mpn)
count = 0
for upc_number in upc:
    if upc_number != 'Does not apply':
        count += 1
        
print "Total UPC's:", count

Total MPN's: 1674
Total UPC's: 753


### Get info about similar items from differrent sellers using MPN number

In [7]:
# import Finding API
from ebaysdk.finding import Connection as finding

# Define API 
api = finding(siteid='EBAY-US', appid= key, config_file=None)

# Get results for similar items
results = []
for position, product_number in enumerate(mpn):
    
    api.execute('findItemsByKeywords', {
        'keywords': product_number,
        'itemFilter': [
            {'name': 'Condition', 'value': 'New'},
            {'paramName': 'Currency', 'paramValue': 'USD'},        
        ],
        'paginationInput': {
            'entriesPerPage': '100',
            'pageNumber': '1'
        },
        'sortOrder': 'CurrentPriceLowest',
        'outputSelector': 'SellerInfo',
        'DetailLevel' : 'ReturnAll'
    })

    result_dict = api.response.dict()
    results.append(result_dict)
    print position

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

### Create a dataframe with collected information

In [74]:
import numpy as np
final_result = []
for pos, result in enumerate(results):
#     print result['paginationOutput']['totalEntries']
    if result['paginationOutput']['totalEntries'] != '0':
        dict_final = {}
        prices = []
        sellers = []
        for element in result['searchResult']['item']:
            if element['sellerInfo']['sellerUserName'] == 'bestframes4you':
                
                dict_final['my_product'] = element['viewItemURL']
                dict_final['my_price'] = element['sellingStatus']['currentPrice']['value']
            item_price = element['sellingStatus']['currentPrice']['value']
            prices.append(float(item_price))
            
            seller = element['sellerInfo']['sellerUserName']
            sellers.append(seller)
            
#         print prices
        if list(set(sellers))[0] == None:
            continue
        else:
        
            sellers_above = 0
            sellers_bellow = 0
            try:
                my_price = float(dict_final['my_price'])
            except KeyError:
                continue

            for price in prices:
                if price > my_price:
                    sellers_above += 1
                elif price < my_price:
                    sellers_bellow += 1
                

    #         print 'sellers_above: ', sellers_above, 'sellers_bellow: ', sellers_bellow
            dict_final['sellers_bellow'] = sellers_bellow
            dict_final['sellers_above'] = sellers_above
            dict_final['average_price'] = str(round(np.mean(prices), 2))
            dict_final['min_price'] = str(np.min(prices))
            dict_final['max_price'] = str(np.max(prices))
            dict_final['total_sellers'] = len(result['searchResult']['item'])
            dict_final['mpn'] = mpn[pos]
            dict_final['upc'] = upc[pos]
            dict_final['item_id'] = productids[pos]
    #         print dict_final
            final_result.append(dict_final)

cols = ['my_product', 'mpn', 'item_id', 'upc', 'my_price', 'total_sellers', 'sellers_bellow', 'sellers_above',\
        'min_price', 'average_price', 'max_price']

final_data = pd.DataFrame.from_dict(final_result)[cols]
final_data.to_csv('final_data.csv')
print 'Final data shape:', final_data.shape

final_data_bellow_my_price = final_data[final_data['sellers_bellow'] > 0].sort_values('mpn')
final_data_bellow_my_price.to_csv('final_data_bellow_my_price.csv')
print final_data_bellow_my_price.shape[0], 'items with price bellow my price'

Final data shape: (1583, 11)
810 items with price bellow my price
